# 1. Configurazione Ambiente
Importiamo le librerie necessarie:
* **Pandas**: Per la gestione dei dati.
* **Wbdata**: Per scaricare i dati macroeconomici dalla Banca Mondiale.
* **Requests & Zipfile**: Per scaricare ed estrarre automaticamente il database delle crisi dal sito dell'FMI.
* **Country_converter**: Per standardizzare i nomi dei paesi.

Configuriamo inoltre i filtri per sopprimere messaggi di avviso non critici.

In [10]:
import pandas as pd
import wbdata
import country_converter as coco
import datetime
import numpy as np
import re
import logging
import warnings
import requests
import zipfile
import io
from google.colab import files

# Configurazione Silenziosa
logging.getLogger('country_converter').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Librerie caricate e ambiente configurato.")

Librerie caricate e ambiente configurato.


# 2. Download Automatico Database Laeven-Valencia (IMF)
Invece di caricare il file manualmente, lo scarichiamo direttamente dai server dell'FMI.
Lo script:
1. Scarica lo ZIP dal link ufficiale.
2. Cerca il file Excel all'interno dell'archivio.
3. Lo carica in memoria pronto per l'analisi.

In [11]:
url_imf = "https://www.imf.org/-/media/websites/imf/imported-datasets/external/pubs/ft/wp/2012/data/_wp12163.zip"

print("Scaricamento database IMF in corso...")

try:
    response = requests.get(url_imf)
    response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Trova il file excel dentro lo zip
        file_names = z.namelist()
        excel_file = [f for f in file_names if f.endswith(('.xlsx', '.xls'))][0]
        print(f"File trovato ed estratto: {excel_file}")

        # Legge il foglio specifico
        with z.open(excel_file) as f:
            df_crisis = pd.read_excel(f, sheet_name='Crisis Resolution and Outcomes')

    print("Dati Crisi caricati in memoria.")

except Exception as e:
    print(f"ERRORE CRITICO: {e}")

Scaricamento database IMF in corso...
File trovato ed estratto: SYSTEMIC BANKING CRISES DATABASE.xlsx
Dati Crisi caricati in memoria.


# 3. Pulizia e Standardizzazione Crisi
Trasformiamo i dati grezzi dell'FMI in un formato utilizzabile:
1. **Paesi**: Convertiamo i nomi in codici ISO3 standard.
2. **Date**: Puliamo la colonna `Start` e `End` usando espressioni regolari (Regex) per gestire note come "ongoing" o date sporche.
3. **Validazione**: Teniamo solo le righe con date e paesi validi.

In [12]:
# Pulizia righe vuote
df_crisis = df_crisis.dropna(subset=['Start'])
df_crisis = df_crisis[df_crisis['Country'].astype(str).str.len() < 50]

# Standardizzazione ISO3
df_crisis['iso3'] = coco.convert(names=df_crisis['Country'].tolist(), to='ISO3', not_found=None)

# Fix liste residue (rimuove aggregati se presenti)
df_crisis['iso3'] = df_crisis['iso3'].apply(lambda x: x if isinstance(x, str) else np.nan)

# Pulizia Anni
def clean_year(val):
    val = str(val).lower()
    if 'ongoing' in val: return 2020
    match = re.search(r'\d{4}', val)
    return int(match.group(0)) if match else np.nan

df_crisis['start_year'] = df_crisis['Start'].apply(clean_year)
df_crisis['end_year'] = df_crisis['End'].apply(clean_year).fillna(df_crisis['start_year'])

df_crisis = df_crisis.dropna(subset=['iso3', 'start_year'])

print(f"Eventi di crisi validati e pronti: {len(df_crisis)}")

Eventi di crisi validati e pronti: 147


In [14]:
df_crisis.head()

,Country,Start,End,Output loss 1/,Fiscal Costs 2/\n(% of GDP),Fiscal cost\n(% of financial sector assets),Peak liquidity 3/,Liquidity support 3/,Peak NPLs 4/,Increase in public debt 5/,Monetary expansion 6/,Credit boom 7/,iso3,start_year,end_year
0,Albania 13/,1994,1994,…,...,...,7.615962,...,26.8,...,...,...,ALB,1994,1994
1,Algeria,1990,1994 11/,41.392253,...,...,37.56032,29.85246,30,19.10056,-4.73,0,DZA,1990,1994
2,Argentina,1980,1982 10/,58.171235,55.1,...,64.64586,62.21225,9,33.14739,10.57,1,ARG,1980,1982
3,Argentina,1989,1991,12.629947,6,36.902775,151.5721,135.665,27,-21.29261,9.98,0,ARG,1989,1991
4,Argentina 12/,1995,1995,0,2,8.579913,71.3528,62.99766,17,8.69467,-0.84,1,ARG,1995,1995
